# Problem Set 3

# Your Name: Yash Manish Raichura

1. Please write clearly! Answer each question in a way that if the code chunks are removed from your document, the result is still readable!
2. Please keep data file in the same folder as your code, and read these w/o any path like "data.csv" (or "./data.csv"). This makes it much easier to check your code!
3. If you use notebooks, upload both the .ipynb and html/pdf file.

### Instructions

The goal of this problem set is to get experience with estimation of causal effects, in particular using the
differences-in-differences (DiD) method. This is a very common task in economics/government/business
analytics. Your task is to estimate the impact of progresa subsidies on the school attendance using the
actual data.
Progresa was a a government social assistance program in Mexico. This program, as well as the details
of its impact, are described in the paper "School subsidies for the poor: evaluating the Mexican Progresa
poverty program", by Paul Shultz (available on Canvas). The data (progresa-sample.csv) is available on
canvas in files/data.
Please read the paper to familiarize yourself with the Progresa program before beginning this problem
set, so you have a rough sense of where the data come from and how they were generated. If you just
proceed into the problem set without understanding Progresa or the data, it will be very difficult! I also
recommend you to consult the section 3.3 in lecture notes.


The timeline of the program was:

Baseline survey conducted in 1997

Intervention begins in 1998, "Wave 1" of data collected in 1998

"Wave 2 of data" collected in 1999

Evaluation ends in 2000, at which point the control villages were treated.

The data are the actual data collected to evaluate the impact of the Progresa program. In this le, each
row corresponds to an observation taken for a given child for a given year. There are two years of data
(1997 and 1998), and just under 40,000 children who are surveyed in both years. For each child-year
observation, the following variables are collected:

In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sp
import operator as op
import statsmodels.formula.api as smf
import sys; print(sys.version)
%matplotlib inline

3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]


In [104]:
df = pd.read_csv('progresa-sample.csv')

In [105]:
df.head(10)

,year,sex,indig,dist_sec,sc,grc,fam_n,min_dist,dist_cap,poor,...,hohedu,hohwag,welfare_index,hohsex,hohage,age,village,folnum,grc97,sc97
0,97,0.0,0.0,4.473,1.0,7.0,7,21.168384,21.168384,pobre,...,6,0.0,583.00000,1.0,35.0,13,163,1,7,1.0
1,98,0.0,0.0,4.473,1.0,8.0,7,21.168384,21.168384,pobre,...,6,0.0,583.00000,1.0,35.0,14,163,1,7,1.0
2,97,1.0,0.0,4.473,1.0,6.0,7,21.168384,21.168384,pobre,...,6,0.0,583.00000,1.0,35.0,12,163,2,6,1.0
3,98,1.0,0.0,4.473,1.0,7.0,7,21.168384,21.168384,pobre,...,6,0.0,583.00000,1.0,35.0,13,163,2,6,1.0
4,97,0.0,0.0,4.473,1.0,2.0,7,21.168384,21.168384,pobre,...,6,0.0,583.00000,1.0,35.0,8,163,3,2,1.0
5,98,0.0,0.0,4.473,1.0,3.0,7,21.168384,21.168384,pobre,...,6,0.0,583.00000,1.0,35.0,9,163,3,2,1.0
6,97,0.0,0.0,3.154,0.0,6.0,6,127.114780,154.196003,pobre,...,4,0.0,684.00000,1.0,85.0,14,271,4,6,0.0
7,98,0.0,0.0,3.154,0.0,6.0,6,127.114780,154.196003,pobre,...,4,0.0,684.00000,1.0,85.0,15,271,4,6,0.0
8,97,1.0,0.0,3.373,1.0,2.0,5,85.300272,105.878669,pobre,...,6,875.0,742.14001,1.0,26.0,9,263,5,2,1.0
9,98,1.0,0.0,3.373,1.0,2.0,5,85.300272,105.878669,pobre,...,6,875.0,742.14001,1.0,26.0,10,263,5,2,1.0


# 1 Descriptive analysis (30pt)
## 1.1 Summary statistics (10pt)
First, learn about data.

### 1. Report summary statistics (mean, standard deviation, and number of missings) for all of the demographic variables in the dataset (i.e., everything except year, folnum, village). A central variable, progresa is coded in a rather unintuitive way. Find it's actual coding scheme. Does this fit with the documentation above? Present these in a single table alphabetized by variable name. Do NOT simply expect the grader to scroll through your output!


The central variable progresa takes 2 values, basal and 0. It is an object type. 'basal' means that it is referring to a treatment group whereas 0 signifies it is referring to a control group. It can be used as a categorical variable instead and values 1 and 0 can be referred for treatment and control groups respectively, or any other values like 'treatment' or 'control' could be assigned instead of a string - basal and an integer - 0.

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77250 entries, 0 to 77249
Data columns (total 21 columns):
year             77250 non-null int64
sex              77226 non-null float64
indig            76950 non-null float64
dist_sec         77250 non-null float64
sc               68797 non-null float64
grc              70701 non-null float64
fam_n            77250 non-null int64
min_dist         77250 non-null float64
dist_cap         77250 non-null float64
poor             77250 non-null object
progresa         77250 non-null object
hohedu           77250 non-null int64
hohwag           77250 non-null float64
welfare_index    77040 non-null float64
hohsex           77230 non-null float64
hohage           77240 non-null float64
age              77250 non-null int64
village          77250 non-null int64
folnum           77250 non-null int64
grc97            77250 non-null int64
sc97             73378 non-null float64
dtypes: float64(12), int64(7), object(2)
memory usage: 12.4+ MB


In [107]:
df.describe()

,year,sex,indig,dist_sec,sc,grc,fam_n,min_dist,dist_cap,hohedu,hohwag,welfare_index,hohsex,hohage,age,village,folnum,grc97,sc97
count,77250.000000,77226.000000,76950.000000,77250.000000,68797.000000,70701.000000,77250.000000,77250.000000,77250.000000,77250.000000,77250.000000,77040.000000,77230.000000,77240.000000,77250.000000,77250.000000,77250.000000,77250.000000,73378.000000
mean,97.500000,0.512211,0.298324,2.418910,0.819818,3.963537,7.215715,103.447520,147.674452,2.768104,586.985312,690.346564,0.925185,44.436717,11.366460,253.614964,19313.000000,3.705372,0.813922
std,0.500003,0.499854,0.457525,2.234109,0.384342,2.499063,2.352900,42.089441,76.063134,2.656106,788.133664,139.491130,0.263095,11.620372,3.167744,149.341967,11150.149239,2.572387,0.389172
min,97.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,9.465392,9.465392,0.000000,0.000000,180.000000,0.000000,15.000000,6.000000,1.000000,1.000000,0.000000,0.000000
25%,97.000000,0.000000,0.000000,0.574000,1.000000,2.000000,6.000000,70.518238,92.327050,0.000000,120.000000,597.000000,1.000000,36.000000,9.000000,126.000000,9657.000000,1.000000,1.000000
50%,97.500000,1.000000,0.000000,2.279000,1.000000,4.000000,7.000000,111.228612,132.001494,2.000000,500.000000,685.000000,1.000000,43.000000,11.000000,257.000000,19313.000000,4.000000,1.000000
75%,98.000000,1.000000,1.000000,3.582000,1.000000,6.000000,9.000000,138.446009,184.445225,4.000000,750.000000,770.000000,1.000000,51.000000,14.000000,385.000000,28969.000000,6.000000,1.000000
max,98.000000,1.000000,1.000000,14.879000,1.000000,14.000000,24.000000,170.457647,359.774457,20.000000,14000.000000,1294.000000,1.000000,98.000000,17.000000,505.000000,38625.000000,14.000000,1.000000


In [108]:
df.isnull().sum()

year                0
sex                24
indig             300
dist_sec            0
sc               8453
grc              6549
fam_n               0
min_dist            0
dist_cap            0
poor                0
progresa            0
hohedu              0
hohwag              0
welfare_index     210
hohsex             20
hohage             10
age                 0
village             0
folnum              0
grc97               0
sc97             3872
dtype: int64

In [109]:
stat_df = df
stat_df = stat_df.drop(['year', 'folnum', 'village'],axis=1)
stat_df.describe().loc[['mean','std']].transpose().sort_index()

,mean,std
age,11.366460,3.167744
dist_cap,147.674452,76.063134
dist_sec,2.418910,2.234109
fam_n,7.215715,2.352900
grc,3.963537,2.499063
grc97,3.705372,2.572387
hohage,44.436717,11.620372
hohedu,2.768104,2.656106
hohsex,0.925185,0.263095
hohwag,586.985312,788.133664


## 1.2 Differences at baseline? (20pt)
Now let's investigate the differences in baseline. Are the baseline (1997) demographic characteristics for
the poor different in treatment and control villages?
### 1. (4pt) Use t-test to determine whether there is a statistically significant difference in the average values of each of the variables in the dataset. Focus only on the data from 1997 for poor.

In [110]:
treatment_df = df[(df.progresa == 'basal')&(df.year == 97)&(df.poor == 'pobre')]

In [111]:
treatment_df

,year,sex,indig,dist_sec,sc,grc,fam_n,min_dist,dist_cap,poor,...,hohedu,hohwag,welfare_index,hohsex,hohage,age,village,folnum,grc97,sc97
6,97,0.0,0.0,3.154,0.0,6.0,6,127.114780,154.196003,pobre,...,4,0.0,684.00000,1.0,85.0,14,271,4,6,0.0
8,97,1.0,0.0,3.373,1.0,2.0,5,85.300272,105.878669,pobre,...,6,875.0,742.14001,1.0,26.0,9,263,5,2,1.0
10,97,0.0,0.0,3.373,1.0,2.0,5,85.300272,105.878669,pobre,...,6,875.0,742.14001,1.0,26.0,7,263,6,2,1.0
12,97,0.0,0.0,3.373,NaN,0.0,5,85.300272,105.878669,pobre,...,6,875.0,742.14001,1.0,26.0,6,263,7,0,NaN
14,97,1.0,1.0,1.935,1.0,2.0,5,127.657608,333.048731,pobre,...,3,500.0,552.00000,1.0,98.0,10,418,8,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77240,97,0.0,1.0,3.148,1.0,2.0,8,137.473203,172.770829,pobre,...,0,500.0,580.00000,1.0,56.0,12,348,38621,2,1.0
77242,97,1.0,1.0,3.148,1.0,1.0,9,137.473203,172.770829,pobre,...,1,500.0,582.50000,1.0,45.0,8,348,38622,1,1.0
77244,97,0.0,1.0,3.148,1.0,2.0,6,137.473203,172.770829,pobre,...,0,0.0,599.00000,0.0,67.0,11,348,38623,2,1.0
77246,97,1.0,1.0,3.148,1.0,1.0,6,137.473203,172.770829,pobre,...,0,0.0,599.00000,0.0,67.0,7,348,38624,1,1.0


In [112]:
control_df = df[(df.progresa != 'basal') & (df.year == 97) & (df.poor == 'pobre')]

In [113]:
control_df

,year,sex,indig,dist_sec,sc,grc,fam_n,min_dist,dist_cap,poor,...,hohedu,hohwag,welfare_index,hohsex,hohage,age,village,folnum,grc97,sc97
0,97,0.0,0.0,4.473,1.0,7.0,7,21.168384,21.168384,pobre,...,6,0.0,583.0,1.0,35.0,13,163,1,7,1.0
2,97,1.0,0.0,4.473,1.0,6.0,7,21.168384,21.168384,pobre,...,6,0.0,583.0,1.0,35.0,12,163,2,6,1.0
4,97,0.0,0.0,4.473,1.0,2.0,7,21.168384,21.168384,pobre,...,6,0.0,583.0,1.0,35.0,8,163,3,2,1.0
184,97,0.0,1.0,1.529,1.0,2.0,5,146.807724,198.916600,pobre,...,0,500.0,637.5,1.0,48.0,9,302,93,2,1.0
186,97,0.0,1.0,1.529,1.0,0.0,5,146.807724,198.916600,pobre,...,0,750.0,530.0,1.0,34.0,6,302,94,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77132,97,0.0,0.0,0.000,1.0,1.0,5,149.957952,149.957952,pobre,...,9,1120.0,747.0,1.0,28.0,7,336,38567,1,1.0
77142,97,1.0,0.0,0.000,1.0,7.0,11,149.957952,149.957952,pobre,...,3,0.0,894.5,1.0,50.0,13,336,38572,7,1.0
77144,97,0.0,0.0,0.000,1.0,5.0,11,149.957952,149.957952,pobre,...,3,0.0,894.5,1.0,50.0,12,336,38573,5,1.0
77146,97,0.0,0.0,0.000,1.0,2.0,11,149.957952,149.957952,pobre,...,3,0.0,894.5,1.0,50.0,8,336,38574,2,1.0


In [114]:
a = treat_df[treat_df.columns.difference(['year','folnum','village'])]

In [115]:
a = a.describe().loc['mean']

In [116]:
a

age               10.716991
dist_cap         150.829074
dist_sec           2.453122
fam_n              7.281327
grc                3.531599
grc97              3.531599
hohage            43.640194
hohedu             2.663139
hohsex             0.924290
hohwag           544.339544
indig              0.324696
min_dist         107.152915
sc                 0.779003
sc97               0.779003
sex                0.519138
welfare_index    655.136672
Name: mean, dtype: float64

In [117]:
target_df = pd.DataFrame({'Variable name':a.index, 'Avg (Treatment villages)':a.values},
                        columns = ['Variable name','Avg (Treatment villages)'])
target_df

,Variable name,Avg (Treatment villages)
0,age,10.716991
1,dist_cap,150.829074
2,dist_sec,2.453122
3,fam_n,7.281327
4,grc,3.531599
5,grc97,3.531599
6,hohage,43.640194
7,hohedu,2.663139
8,hohsex,0.924290
9,hohwag,544.339544


In [118]:
b = control_df[control_df.columns.difference(['year','folnum','village'])]
b = b.describe().loc['mean']
target_df2 = pd.DataFrame({'Variable name':b.index, 'Avg (control)':b.values},
                        columns = ['Variable name','Avg (control)'])
target_df2

merged_df=pd.merge(target_df,target_df2, on = 'Variable name')
merged_df

,Variable name,Avg (Treatment villages),Avg (control)
0,age,10.716991,10.742023
1,dist_cap,150.829074,153.769730
2,dist_sec,2.453122,2.507662
3,fam_n,7.281327,7.302469
4,grc,3.531599,3.543050
5,grc97,3.531599,3.543050
6,hohage,43.640194,44.276918
7,hohedu,2.663139,2.590348
8,hohsex,0.924290,0.922947
9,hohwag,544.339544,573.163558


In [119]:
tcomp = {}
treat_df = treat_df[treat_df.columns.difference(['year','folnum','village', 'poor', 'progresa'])].fillna(0)
control_df = control_df[control_df.columns.difference(['year','folnum','village', 'poor', 'progresa'])].fillna(0)
list_val = list(treat_df)
for x in range(0,len(final_df)):
    tcomp.update({list_val[x]: sp.ttest_ind(treat_df.transpose().iloc[x],control_df.transpose().iloc[x]).pvalue})

list_df = pd.DataFrame(list(tcomp.items()), columns = ['Variable name','p-value'])
list_df

merged_df=pd.merge(merged_df,tlist_df, how='left' ,on = 'Variable name')
merged_df

,Variable name,Avg (Treatment villages),Avg (control),p-value
0,age,10.716991,10.742023,4.785594e-01
1,dist_cap,150.829074,153.769730,8.415005e-04
2,dist_sec,2.453122,2.507662,3.569843e-02
3,fam_n,7.281327,7.302469,4.271039e-01
4,grc,3.531599,3.543050,6.890151e-01
5,grc97,3.531599,3.543050,6.890151e-01
6,hohage,43.640194,44.276918,1.515918e-06
7,hohedu,2.663139,2.590348,1.105093e-02
8,hohsex,0.924290,0.922947,6.217737e-01
9,hohwag,544.339544,573.163558,3.253835e-04


In [120]:
merged_df['difference'] = merged_df['Avg (Treatment villages)'] - merged_df['Avg (control)'] 

In [121]:
merged_df

,Variable name,Avg (Treatment villages),Avg (control),p-value,difference
0,age,10.716991,10.742023,4.785594e-01,-0.025032
1,dist_cap,150.829074,153.769730,8.415005e-04,-2.940656
2,dist_sec,2.453122,2.507662,3.569843e-02,-0.054540
3,fam_n,7.281327,7.302469,4.271039e-01,-0.021142
4,grc,3.531599,3.543050,6.890151e-01,-0.011450
5,grc97,3.531599,3.543050,6.890151e-01,-0.011450
6,hohage,43.640194,44.276918,1.515918e-06,-0.636724
7,hohedu,2.663139,2.590348,1.105093e-02,0.072791
8,hohsex,0.924290,0.922947,6.217737e-01,0.001343
9,hohwag,544.339544,573.163558,3.253835e-04,-28.824015


### 3. (4pt) Do you find any statistically significant differences between treatment and control villages as baseline?

3. We assume the significane value to be 0.05 in this case. We see that for variables like sex, dist_sec, min_dist, dist_cap, hohedu, hohwag,hohage and welfare_index have a p value of less than 0.05 and hence are statistically significant.

### 4. (4pt) Why does it matter if there are differences at baseline?

4. The control and treatment groups have have different averages for almost all the values and any changes directed towards the treatment group due to the treatment cannot be guaranteed that it was a result of the treatment since all the values between the groups were already different.

### 5. (4pt) What does this imply about how to measure the impact of the treatment?

5. We cannot use before and after estimators to check for causal effect since there is a difference between the values of the variable in control and treatment groups. We need to use the difference in differences estimator to check for causal effect.

# 2 Measuring Impact
Our goal is to estimate the causal impact of the Progresa program on the schooling outcomes of individuals
in Mexico. We will focus on the impact of the program on the poor, since only the poor were eligible to
receive the Progresa assistance.

## 2.1 Before-after estimator (10pt)
First, implement the before-after estimator. Compare the schooling rate of poor households in progresa
villages before (i.e. 1997) and after (i.e. 1998) the program

### 1. (2pt) compute the estimator by just comparing the average schooling rates for these villages

Removing the na values, cleaning the dataset and using the data before the year 1998

False: ID for year 1998 with 0.834 being the average enrollment rate
True: ID for year 1997 with 0.81 being the average enrollment rate



In [122]:
copy_df = df.copy()
copy_df=copy_df.dropna()
copy_df['before'] =copy_df.year<98
copy_df.before.describe()

copy_df[copy_df.poor=='pobre'].groupby('before').sc.mean()

before
False    0.834793
True     0.819800
Name: sc, dtype: float64

#### 2. (3pt) now re-compute the estimator using linear regression, and individual schooling rates. Do not include other regressors.

In [123]:
model = smf.ols(formula='sc~before',data=copy_df[copy_df.poor=='pobre'])
fit1=model.fit()
print(fit1.summary())

                            OLS Regression Results                            
Dep. Variable:                     sc   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     22.18
Date:                Wed, 05 Feb 2020   Prob (F-statistic):           2.49e-06
Time:                        05:55:36   Log-Likelihood:                -25445.
No. Observations:               56893   AIC:                         5.089e+04
Df Residuals:                   56891   BIC:                         5.091e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.8348      0.002    356.

#### 3. (3pt) fnally, estimate a multiple regression model that includes other covariates

In [124]:
model = smf.ols(formula='sc~before+sex+ indig+ dist_sec+grc+ fam_n+ min_dist+dist_cap+progresa+ hohedu+ hohwag+ welfare_index+hohsex+ hohage+ age+ grc97+sc97',data=copy_df[copy_df.poor=='pobre'])
fit2 = model.fit()
print(fit2.summary())

                            OLS Regression Results                            
Dep. Variable:                     sc   R-squared:                       0.716
Model:                            OLS   Adj. R-squared:                  0.716
Method:                 Least Squares   F-statistic:                     8448.
Date:                Wed, 05 Feb 2020   Prob (F-statistic):               0.00
Time:                        05:55:37   Log-Likelihood:                 10384.
No. Observations:               56893   AIC:                        -2.073e+04
Df Residuals:                   56875   BIC:                        -2.057e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.3121      0.01

#### 4. (2pt) compare all the estimators. Are your estimates statistically signifcant? What do they suggest about the effcacy of the progresa program?

Here, we see that the response variable enrollment in village has no dependency on the year before 1998 and hence the R-squared value is also 0.
In multivariate regression, we notice that enrollment has many predictor variables with p-values less than 0.05 making them statistically significant. These predictor variables are -  before[T.True], progresa[T.basal], sex , indig, dist_sec, grc, min_dist, dist_camp, hohedu, hohsex, age, grc97, sc97.
We see a positive coefficient of 0.0155 for progresa[T.basal] which indicates that the treatment has been effective but it also depends on other factors and the results of effective treatment cannot be directed only towards the progresa program.

## 2.2 Cross-sectional estimator (10pt)
Now let's implement the cross-sectional estimator. Proceed along the same lines as what you did above.
### 1. (2pt) Begin by estimating the impact of Progresa by compring the average enrollment rate among poor households in the treatment villages and the average enrollment rate among poor households in the control villages. What do you find?


In [125]:
avg_treatment=copy_df[(newdata['poor']=='pobre') & (copy_df.progresa=='basal')& (copy_df.year==98)]
print("Average enrollment in treatment village",avg_treatment['sc'].mean())

avg_control=copy_df[(newdata.poor=='pobre')&(copy_df.progresa!='basal')&(copy_df.year==98)]
print("Average enrollment in control village",avg_control['sc'].mean())

Average enrollment in treatment village 0.849257030578411
Average enrollment in control village 0.810923092511906


The results above suggest that average enrollment in treatment village was higher as compared to control village.

### 2. (3pt) Now repeat the estimator using simple regression.

In [126]:
copy_df['after']=copy_df.year>97
copy_df.after.describe()

model = smf.ols(formula='sc~progresa',data=copy_df[(copy_df.poor=='pobre') & (copy_df.year>97)])
fit3 = model.fit()
print(fit3.summary())

                            OLS Regression Results                            
Dep. Variable:                     sc   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     65.64
Date:                Wed, 05 Feb 2020   Prob (F-statistic):           5.66e-16
Time:                        05:55:37   Log-Likelihood:                -11171.
No. Observations:               26155   AIC:                         2.235e+04
Df Residuals:                   26153   BIC:                         2.236e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8109      0.00

### 3. (3pt) Third, use multiple regression to get the same estimate

In [127]:
model = smf.ols(formula='sc~progresa+sex+ indig+ dist_sec+grc+ fam_n+ min_dist+dist_cap+ hohedu+ hohwag+ welfare_index+hohsex+ hohage+ age+ grc97+sc97',data=copy_df[(copy_df.poor=='pobre')& (copy_df.year>97)])
fit4 = model.fit()
print(fit4.summary())

                            OLS Regression Results                            
Dep. Variable:                     sc   R-squared:                       0.452
Model:                            OLS   Adj. R-squared:                  0.451
Method:                 Least Squares   F-statistic:                     1345.
Date:                Wed, 05 Feb 2020   Prob (F-statistic):               0.00
Time:                        05:55:37   Log-Likelihood:                -3347.5
No. Observations:               26155   AIC:                             6729.
Df Residuals:                   26138   BIC:                             6868.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.7796      0.02

### 4. (2pt) Finally, as above, compare your three estimators. What do you find? Are the effects statistically significant?


With cross sectional estimator we see that the coefficient of progresa[T.basal] is higher than the before-after estimator suggesting that there is higher enrollment in the treatment group.
It also shows that other predictor variables have lower statistical significance and that the response enrollment does not depend on those. These variables are fam_n, hohwag, welfare_index, hohsex and hoh_age. Whereas, on the other hand, before-after estimator suggested that hohsex is a statistically significant value.
This just shows that the enrollment depended more on the treatment.

## 2.3 Differences-in-differences estimator (30pt)
Now we are ready for DiD estimator. Proceed along the same lines as above.

### 1. (6pt) Start with the simple table. However, DiD requires 4-way comparison. So compare the average enrollment rate among poor households in the treatment villages and the average enrollment rate among poor households in the control villages, both 1997 and 1998. What do you find?


In [128]:
df_97 = df[(df['poor'] == 'pobre') & (df['year'] == 97)]
df_98 = df[(df['poor'] == 'pobre') & (df['year'] == 98)]


treatment98 = df_98['sc'][df_98.progresa == "basal"].mean() 
treatment97 = df_97['sc'][df_97.progresa == "basal"].mean()
control98 =  df_98['sc'][df_98.progresa != "basal"].mean()
control97 =  df_97['sc'][df_97.progresa != "basal"].mean()

diffindiff = pd.DataFrame(index = ['Avg Enrollment Before Treatment', 'Avg Enrollment After Treatment','Difference in Difference'],
                           columns = ['Control Group', 'Treatment Group', 'Difference in Difference'])
diffindiff.loc["Avg Enrollment Before Treatment","Control Group"] = control97
diffindiff.loc["Avg Enrollment After Treatment","Control Group"] = control98
diffindiff.loc["Avg Enrollment Before Treatment","Treatment Group"] = treatment97
diffindiff.loc["Avg Enrollment After Treatment","Treatment Group"] = treatment98
diffindiff.loc["Difference in Difference","Difference in Difference"] = (treatment98 - treatment97) - (control98 - control97)
diffindiff.fillna('')
diffindiff

,Control Group,Treatment Group,Difference in Difference
Avg Enrollment Before Treatment,0.815186,0.822697,NaN
Avg Enrollment After Treatment,0.807637,0.846479,NaN
Difference in Difference,NaN,NaN,0.0313313


The above result displays enrollment rates for both the control and treatment groups and also takes into consideration, the years 1997 and 1998.
We see that enrollment after treatment in the treatment group is higher than both the previous estimators suggested.

### 2. (10pt) Now repeat the estimator using simple regression

In [129]:
new_df=df.copy()
new_df.loc[new_df.year == 97, 'treat'] = 0
new_df.loc[new_df.year == 98, 'treat'] = 1
pobre_df = new_df[newdata1.poor == 'pobre']

In [130]:
#Difference in difference estimator without any controls
model  = smf.ols(formula='sc ~ progresa*treat', data=pobre_df)
fit5 = model.fit()
fit5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     28.31
Date:                Wed, 05 Feb 2020   Prob (F-statistic):           2.76e-18
Time:                        05:55:38   Log-Likelihood:                -26242.
No. Observations:               58372   AIC:                         5.249e+04
Df Residuals:                   58368   BIC:                         5.253e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   0.8152      0.003    233.182      0.000       0.808       0.822
progresa[T.basal]           0.0075      0.004      1.691      0.091      -0.001       0.016
treat                      -0.0075      0.005     -1.480      0.139      -0.018       0.002
progresa[T.basal]:treat     0.0313      0.006      4.835      0.000       0.019       0.044
==============================================================================
Omnibus:                    15346.988   Durbin-Watson:                   1.397
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30608.651
Skew:                          -1.711   Prob(JB):                         0.00
Kurtosis:                       3.937   Cond. No.                         7.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### 3. (8pt) And as above, use multiple regression to get the same estimate

In [131]:
model = smf.ols(formula='sc ~ progresa*treat +sex+ indig+ dist_sec+grc+ fam_n+ min_dist+dist_cap+ hohedu+ hohwag+ welfare_index+hohsex+ hohage+ age+ grc97+sc97', data=pobre_df)
fit6 = model.fit()
fit6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     sc   R-squared:                       0.717
Model:                            OLS   Adj. R-squared:                  0.717
Method:                 Least Squares   F-statistic:                     7996.
Date:                Wed, 05 Feb 2020   Prob (F-statistic):               0.00
Time:                        05:55:38   Log-Likelihood:                 10429.
No. Observations:               56893   AIC:                        -2.082e+04
Df Residuals:                   56874   BIC:                        -2.065e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   0.3405      0.010     34.657      0.000       0.321       0.360
progresa[T.basal]           0.0003      0.002      0.132      0.895      -0.004       0.005
treat                      -0.0398      0.003    -13.335      0.000      -0.046      -0.034
progresa[T.basal]:treat     0.0331      0.003      9.471      0.000       0.026       0.040
sex                         0.0071      0.002      4.189      0.000       0.004       0.010
indig                       0.0046      0.002      2.192      0.028       0.000       0.009
dist_sec                   -0.0024      0.000     -6.010      0.000      -0.003      -0.002
grc                         0.0081      0.002      4.967      0.000       0.005       0.011
fam_n                       0.0002      0.000      0.510      0.610      -0.001       0.001
min_dist                 9.708e-05   2.79e-05      3.479      0.001    4.24e-05       0.000
dist_cap                 5.857e-05   1.62e-05      3.626      0.000    2.69e-05    9.02e-05
hohedu                      0.0019      0.000      4.919      0.000       0.001       0.003
hohwag                  -1.144e-06   1.22e-06     -0.935      0.350   -3.54e-06    1.26e-06
welfare_index           -6.664e-06   7.98e-06     -0.835      0.403   -2.23e-05    8.97e-06
hohsex                      0.0072      0.003      2.146      0.032       0.001       0.014
hohage                   9.679e-06   8.61e-05      0.112      0.910      -0.000       0.000
age                        -0.0200      0.001    -32.918      0.000      -0.021      -0.019
grc97                      -0.0041      0.002     -2.484      0.013      -0.007      -0.001
sc97                        0.7952      0.003    284.662      0.000       0.790       0.801
==============================================================================
Omnibus:                    22880.238   Durbin-Watson:                   2.140
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           536409.048
Skew:                          -1.397   Prob(JB):                         0.00
Kurtosis:                      17.781   Cond. No.                     1.23e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.23e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 4. (6pt) Finally, as above, compare your three estimators. What do you find? Are the effects statistically significant?


For treatment group after 1998, the estimate of progresa treatment effect is 0.0314., the average enrollment rate is increased by 0.0274 with a standard error of 0.004 after 1998 The progresa co-efficient is below 0.05 and is not significant this time.

## 2.4 Compare the estimators (20pt)
Now you have used three estimators to assess the effect of Progresa program.

### 1. (10pt) List the identifying assumptions (counterfactual assumptions) behind all three models. Which ones do you find more/less plausible?

The counterfactual assumption is that the changes in treatment group and control group would be exactly the same if progressa treatment was not present.

### 2. (10pt) Compare the estimates of all three models. Do your analysis suggest that progresa program had a positive impact on schooling rates?

From linear regression models, we can say that progressa improved the enrollment rate im both treatment and control groups. Diff in Diff estimator helps us take the differences in the baseline into consideration. This estimator also suggests that Progressa had a positive effect on enrollment. Diff-in-Diff estimator predicted more accurate results and suggests that there was a causal relationship.
Hence, we can say that progressa program had a positive impact on schooling rates but the impact was not very high.